In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import numpy as np
import pandas as pd
import os

In [14]:
os.chdir("/content/drive/My Drive/data")
dat = pd.read_csv("dat_final1초전.csv", encoding='CP949')
data = dat.iloc[:300,:5]

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   y           300 non-null    float64
 1   date        300 non-null    object 
 2   temp        300 non-null    float64
 3   rain        300 non-null    float64
 4   wind_speed  300 non-null    float64
dtypes: float64(4), object(1)
memory usage: 11.8+ KB


In [16]:
! pip install catboost

In [17]:
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=1000,
                          loss_function='RMSE',
                          eval_metric='RMSE'
                          ,task_type="GPU"
                          ,logging_level = 'Silent'
)

In [18]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [19]:
def rmse(true, pred):
    return np.sqrt(((pred - true) ** 2).mean())

In [20]:
def cv_cat(train):
  result = []
  X = train[train.columns.difference(['y'])]
  y = train.loc[:, 'y']
  for k, (train_index,test_index) in enumerate(kf.split(data)):
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index, :], X.iloc[test_index,:]
    cat_fit = cb_model.fit(X_train, y_train, cat_features=list(train.select_dtypes(include=['object', 'int64']).columns))
    pred = cat_fit.predict(X_valid)
    true = y_valid.to_numpy().flatten()
    error = rmse(true, pred)
    result.append(error)
  return (np.mean(result)) 


In [21]:
from itertools import combinations

In [55]:
feature_list = list(set(data.columns.tolist())-set('y'))
used_feature = []
unused_feature = feature_list
num_feature = 1

while num_feature =< data.shape[1]:
  variable_df = pd.DataFrame()
  if (num_feature>2):
    var_comb1 = list(combinations(used_feature,num_feature-2))
  else: 
    var_comb1 = []
  if (num_feature>1):
    var_comb2 = list((used_feature,))
  else: 
    var_comb2 = [] 
  var_comb3 = [t+tuple(used_feature) for t in list(combinations(unused_feature, 1))]

  variable_df['variable'] = var_comb1 + var_comb2+ var_comb3
  variable_df['rmse'] = 0

  for k in range(variable_df.shape[0]):
    using_var = list(variable_df.iloc[k,0]+('y',))
    variable_df.iloc[k,1] = cv_cat(data.loc[:, using_var])
  
  idx = [int(variable_df['rmse'].idxmin()),]
  min_variable = variable_df.iloc[idx].iloc[0,0]
  if (len(min_variable)==num_feature-1):
    print('best variable:', min_variable)
    break
  else:
    num_feature = len(min_variable)+1
    used_feature = min_variable
    unused_feature = list(set(feature_list)-set(used_feature))
    print(num_feature, used_feature)

2
3
best variable: ('date', 'temp')


In [56]:
variable_df

,variable,rmse
0,"(date,)",26.710858
1,"(temp,)",25.455858
2,"(date, temp)",24.031207
3,"(wind_speed, date, temp)",24.122238
4,"(rain, date, temp)",24.135632
